# Settings

In [ ]:

import os
import sys
import subprocess
import wave
import aifc
import math
import audioop
import collections
import json
import base64
import threading
import platform
import stat
import hashlib
import hmac
import time
import uuid

In [1]:
import io
import pyaudio

In [34]:
import wave

# Source

In [2]:
audio = pyaudio.PyAudio()

In [3]:
count = audio.get_device_count()

In [4]:
audio.get_device_info_by_index(0)

{'index': 0,
 'structVersion': 2,
 'name': 'Built-in Microphone',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.0029478458049886623,
 'defaultLowOutputLatency': 0.01,
 'defaultHighInputLatency': 0.01310657596371882,
 'defaultHighOutputLatency': 0.1,
 'defaultSampleRate': 44100.0}

In [5]:
device_info = audio.get_device_info_by_index(0)
sample_rate = int(device_info["defaultSampleRate"])

In [6]:
#audio.terminate()

In [7]:
device_index = 0

In [8]:
form = pyaudio.paInt16

In [9]:
SAMPLE_WIDTH = pyaudio.get_sample_size(form)

In [10]:
SAMPLE_RATE = sample_rate

In [11]:
CHUNK = 1024

In [12]:
#audio = None
stream = None

In [13]:
class MicrophoneStream(object):
    def __init__(self, pyaudio_stream):
        self.pyaudio_stream = pyaudio_stream

    def read(self, size):
        return self.pyaudio_stream.read(size, exception_on_overflow=False)

    def close(self):
        try:
            # sometimes, if the stream isn't stopped, closing the stream throws an exception
            if not self.pyaudio_stream.is_stopped():
                self.pyaudio_stream.stop_stream()
        finally:
            self.pyaudio_stream.close()

In [38]:
stream = MicrophoneStream(
    audio.open(
        input_device_index=device_index, channels=1, format=form,
        rate=SAMPLE_RATE, frames_per_buffer=CHUNK, input=True,
    )
)

In [31]:
stream.close()

# Recogniser

In [16]:
energy_threshold = 300
dynamic_energy_treshold = True
dynamic_energy_adjustment_damping = 0.15
dynamic_energy_ratio = 1.5
pause_threshold = 0.8  # seconds of non-speaking audio before a phrase is considered complete
operation_timeout = None  # seconds after an internal operation (e.g., an API request) starts before it times out, or ``None`` for no timeout

phrase_threshold = 0.3  # minimum seconds of speaking audio before we consider the speaking audio a phrase - values below this are ignored (for filtering out clicks and pops)
non_speaking_duration = 0.5  # seconds of non-speaking audio to keep on both sides of the recording

In [47]:
def record(duration=None, offset=None):
    frames = io.BytesIO()
    seconds_per_buffer = (CHUNK + 0.0) / SAMPLE_RATE
    elapsed_time = 0
    offset_time = 0
    offset_reached = False
    while True:  # loop for the total number of chunks needed
        if offset and not offset_reached:
            offset_time += seconds_per_buffer
            if offset_time > offset:
                offset_reached = True

        buffer = stream.read(CHUNK)
        if len(buffer) == 0: break
    
        print(elapsed_time)
        if offset_reached or not offset:
            elapsed_time += seconds_per_buffer
            if duration and elapsed_time > duration: 
                break

            frames.write(buffer)

    frame_data = frames.getvalue()
    frames.close()
    #return frame_data
    return AudioData(frame_data, SAMPLE_RATE, SAMPLE_WIDTH)

In [58]:
bla = record(2)

0
0.023219954648526078
0.046439909297052155
0.06965986394557823
0.09287981859410431
0.11609977324263039
0.13931972789115646
0.16253968253968254
0.18575963718820862
0.2089795918367347
0.23219954648526078
0.25541950113378686
0.2786394557823129
0.30185941043083897
0.325079365079365
0.3482993197278911
0.37151927437641713
0.3947392290249432
0.41795918367346924
0.4411791383219953
0.46439909297052134
0.4876190476190474
0.5108390022675735
0.5340589569160996
0.5572789115646257
0.5804988662131518
0.6037188208616779
0.626938775510204
0.6501587301587302
0.6733786848072563
0.6965986394557824
0.7198185941043085
0.7430385487528346
0.7662585034013607
0.7894784580498868
0.8126984126984129
0.835918367346939
0.8591383219954651
0.8823582766439912
0.9055782312925174
0.9287981859410435
0.9520181405895696
0.9752380952380957
0.9984580498866218
1.021678004535148
1.044897959183674
1.0681179138322001
1.0913378684807262
1.1145578231292523
1.1377777777777784
1.1609977324263046
1.1842176870748307
1.2074376417233568

In [32]:
bla

In [59]:
wv = bla.get_wav_data()

In [60]:
wv

b'RIFF$\xb0\x02\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00D\xac\x00\x00\x88X\x01\x00\x02\x00\x10\x00data\x00\xb0\x02\x00"\x004\x005\x00>\x00A\x00@\x00L\x00T\x00^\x00o\x00~\x00\x8f\x00\xa5\x00\xb0\x00\xbe\x00\xd4\x00\xda\x00\xea\x00\xf8\x00\x00\x01\x13\x01\x1b\x01\'\x01+\x01$\x01 \x01\x19\x01\x10\x01\x02\x01\xf5\x00\xeb\x00\xe3\x00\xe0\x00\xda\x00\xda\x00\xdb\x00\xdd\x00\xe3\x00\xd9\x00\xcd\x00\xb9\x00\xa0\x00\x80\x00O\x00!\x00\xf9\xff\xcd\xff\xa3\xff\x82\xffj\xffa\xffd\xffm\xffz\xff\x85\xff\x90\xff\xa0\xff\xa1\xff\x90\xff\x84\xffv\xfff\xff[\xffT\xff\\\xffj\xff\x81\xff\x9b\xff\xae\xff\xb7\xff\xb5\xff\xaa\xff\x95\xffs\xffK\xff.\xff\x0c\xff\xef\xfe\xdf\xfe\xcd\xfe\xc9\xfe\xcc\xfe\xcb\xfe\xc8\xfe\xb1\xfe\x9c\xfe\x83\xfec\xfeE\xfe0\xfe;\xfek\xfe\xc2\xfe5\xff\xb3\xffK\x00\xf8\x00\x9e\x01\x15\x02c\x02\x98\x02\xba\x02\xc7\x02\xc6\x02\xd1\x02\xed\x02 \x03y\x03\xd9\x035\x04\x9f\x04\xf3\x04\x1d\x05\x13\x05\xcb\x04c\x04\xd3\x03\x1f\x03h\x02\xad\x01\xfd\x00r\x00\xfa\xff\x98\xff\\\xff%\xff\xf3\xfe\

### Test ouput by writing it to wav file

In [61]:
with open('bla.wav', 'wb') as output:
    output.write(wv)

In [46]:
class AudioData(object):
    """
    Creates a new ``AudioData`` instance, which represents mono audio data.
    The raw audio data is specified by ``frame_data``, which is a sequence of bytes representing audio samples. This is the frame data structure used by the PCM WAV format.
    The width of each sample, in bytes, is specified by ``sample_width``. Each group of ``sample_width`` bytes represents a single audio sample.
    The audio data is assumed to have a sample rate of ``sample_rate`` samples per second (Hertz).
    Usually, instances of this class are obtained from ``recognizer_instance.record`` or ``recognizer_instance.listen``, or in the callback for ``recognizer_instance.listen_in_background``, rather than instantiating them directly.
    """
    def __init__(self, frame_data, sample_rate, sample_width):
        assert sample_rate > 0, "Sample rate must be a positive integer"
        assert sample_width % 1 == 0 and 1 <= sample_width <= 4, "Sample width must be between 1 and 4 inclusive"
        self.frame_data = frame_data
        self.sample_rate = sample_rate
        self.sample_width = int(sample_width)

    def get_segment(self, start_ms=None, end_ms=None):
        """
        Returns a new ``AudioData`` instance, trimmed to a given time interval. In other words, an ``AudioData`` instance with the same audio data except starting at ``start_ms`` milliseconds in and ending ``end_ms`` milliseconds in.
        If not specified, ``start_ms`` defaults to the beginning of the audio, and ``end_ms`` defaults to the end.
        """
        assert start_ms is None or start_ms >= 0, "``start_ms`` must be a non-negative number"
        assert end_ms is None or end_ms >= (0 if start_ms is None else start_ms), "``end_ms`` must be a non-negative number greater or equal to ``start_ms``"
        if start_ms is None:
            start_byte = 0
        else:
            start_byte = int((start_ms * self.sample_rate * self.sample_width) // 1000)
        if end_ms is None:
            end_byte = len(self.frame_data)
        else:
            end_byte = int((end_ms * self.sample_rate * self.sample_width) // 1000)
        return AudioData(self.frame_data[start_byte:end_byte], self.sample_rate, self.sample_width)

    def get_raw_data(self, convert_rate=None, convert_width=None):
        """
        Returns a byte string representing the raw frame data for the audio represented by the ``AudioData`` instance.
        If ``convert_rate`` is specified and the audio sample rate is not ``convert_rate`` Hz, the resulting audio is resampled to match.
        If ``convert_width`` is specified and the audio samples are not ``convert_width`` bytes each, the resulting audio is converted to match.
        Writing these bytes directly to a file results in a valid `RAW/PCM audio file <https://en.wikipedia.org/wiki/Raw_audio_format>`__.
        """
        assert convert_rate is None or convert_rate > 0, "Sample rate to convert to must be a positive integer"
        assert convert_width is None or (convert_width % 1 == 0 and 1 <= convert_width <= 4), "Sample width to convert to must be between 1 and 4 inclusive"

        raw_data = self.frame_data

        # make sure unsigned 8-bit audio (which uses unsigned samples) is handled like higher sample width audio (which uses signed samples)
        if self.sample_width == 1:
            raw_data = audioop.bias(raw_data, 1, -128)  # subtract 128 from every sample to make them act like signed samples

        # resample audio at the desired rate if specified
        if convert_rate is not None and self.sample_rate != convert_rate:
            raw_data, _ = audioop.ratecv(raw_data, self.sample_width, 1, self.sample_rate, convert_rate, None)

        # convert samples to desired sample width if specified
        if convert_width is not None and self.sample_width != convert_width:
            if convert_width == 3:  # we're converting the audio into 24-bit (workaround for https://bugs.python.org/issue12866)
                raw_data = audioop.lin2lin(raw_data, self.sample_width, 4)  # convert audio into 32-bit first, which is always supported
                try: audioop.bias(b"", 3, 0)  # test whether 24-bit audio is supported (for example, ``audioop`` in Python 3.3 and below don't support sample width 3, while Python 3.4+ do)
                except audioop.error:  # this version of audioop doesn't support 24-bit audio (probably Python 3.3 or less)
                    raw_data = b"".join(raw_data[i + 1:i + 4] for i in range(0, len(raw_data), 4))  # since we're in little endian, we discard the first byte from each 32-bit sample to get a 24-bit sample
                else:  # 24-bit audio fully supported, we don't need to shim anything
                    raw_data = audioop.lin2lin(raw_data, self.sample_width, convert_width)
            else:
                raw_data = audioop.lin2lin(raw_data, self.sample_width, convert_width)

        # if the output is 8-bit audio with unsigned samples, convert the samples we've been treating as signed to unsigned again
        if convert_width == 1:
            raw_data = audioop.bias(raw_data, 1, 128)  # add 128 to every sample to make them act like unsigned samples again

        return raw_data

    def get_wav_data(self, convert_rate=None, convert_width=None):
        """
        Returns a byte string representing the contents of a WAV file containing the audio represented by the ``AudioData`` instance.
        If ``convert_width`` is specified and the audio samples are not ``convert_width`` bytes each, the resulting audio is converted to match.
        If ``convert_rate`` is specified and the audio sample rate is not ``convert_rate`` Hz, the resulting audio is resampled to match.
        Writing these bytes directly to a file results in a valid `WAV file <https://en.wikipedia.org/wiki/WAV>`__.
        """
        raw_data = self.get_raw_data(convert_rate, convert_width)
        sample_rate = self.sample_rate if convert_rate is None else convert_rate
        sample_width = self.sample_width if convert_width is None else convert_width

        # generate the WAV file contents
        with io.BytesIO() as wav_file:
            wav_writer = wave.open(wav_file, "wb")
            try:  # note that we can't use context manager, since that was only added in Python 3.4
                wav_writer.setframerate(sample_rate)
                wav_writer.setsampwidth(sample_width)
                wav_writer.setnchannels(1)
                wav_writer.writeframes(raw_data)
                wav_data = wav_file.getvalue()
            finally:  # make sure resources are cleaned up
                wav_writer.close()
        return wav_data

    def get_aiff_data(self, convert_rate=None, convert_width=None):
        """
        Returns a byte string representing the contents of an AIFF-C file containing the audio represented by the ``AudioData`` instance.
        If ``convert_width`` is specified and the audio samples are not ``convert_width`` bytes each, the resulting audio is converted to match.
        If ``convert_rate`` is specified and the audio sample rate is not ``convert_rate`` Hz, the resulting audio is resampled to match.
        Writing these bytes directly to a file results in a valid `AIFF-C file <https://en.wikipedia.org/wiki/Audio_Interchange_File_Format>`__.
        """
        raw_data = self.get_raw_data(convert_rate, convert_width)
        sample_rate = self.sample_rate if convert_rate is None else convert_rate
        sample_width = self.sample_width if convert_width is None else convert_width

        # the AIFF format is big-endian, so we need to covnert the little-endian raw data to big-endian
        if hasattr(audioop, "byteswap"):  # ``audioop.byteswap`` was only added in Python 3.4
            raw_data = audioop.byteswap(raw_data, sample_width)
        else:  # manually reverse the bytes of each sample, which is slower but works well enough as a fallback
            raw_data = raw_data[sample_width - 1::-1] + b"".join(raw_data[i + sample_width:i:-1] for i in range(sample_width - 1, len(raw_data), sample_width))

        # generate the AIFF-C file contents
        with io.BytesIO() as aiff_file:
            aiff_writer = aifc.open(aiff_file, "wb")
            try:  # note that we can't use context manager, since that was only added in Python 3.4
                aiff_writer.setframerate(sample_rate)
                aiff_writer.setsampwidth(sample_width)
                aiff_writer.setnchannels(1)
                aiff_writer.writeframes(raw_data)
                aiff_data = aiff_file.getvalue()
            finally:  # make sure resources are cleaned up
                aiff_writer.close()
        return aiff_data